In [7]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import os
# import seaborn as sns
from datetime import datetime
import re

In [69]:
folder = "../Data"
files = ["round1-2.json", "allrounds.json", "round1.json", "zumbi-venom-11-07-session2.json"]
# files = ["round1.json"]
sessions = {}
for file in files:
    path = os.path.join(folder, file)
    raw_data = pd.json_normalize(pd.read_json(path)['execution_stream'])
    session_key = file.split('.')[0]
    sessions[session_key] = raw_data

In [77]:
for key in sessions:
    print(f"{key} -\n first_message: {sessions[key].head(1)['topic']} -\n last_message: {sessions[key].tail(1)['topic']}\n ------")

round1-2 -
 first_message: 0    2019-05-18T13:35:43.454Z-7160cfbc-e6f1-d8e5-5c...
Name: topic, dtype: object -
 last_message: 33263    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
allrounds -
 first_message: 0    2019-05-18T20:06:23.809Z-6988c9c5-b742-72b1-83...
Name: topic, dtype: object -
 last_message: 102645    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
round1 -
 first_message: 0    2019-05-18T13:35:43.454Z-7160cfbc-e6f1-d8e5-5c...
Name: topic, dtype: object -
 last_message: 19903    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
zumbi-venom-11-07-session2 -
 first_message: 0    2019-07-11T14:20:52.342Z-a5b70eec-48b3-2044-55...
Name: topic, dtype: object -
 last_message: 10185    data/case/148e8e25-5f91-4189-83cd-646c1ebed299...
Name: topic, dtype: object
 ------


## Os arquivos de backup mais antigos são de 18-05-2019 apenas o *zumbi-venom-11-07-session2* inclui o férias mas essa sessão também se encontra no *executions-bkp-30-08-19.json*

In [110]:
bkp_filename = "executions-bkp-30-08-19.jsonl"
path = os.path.join(folder, bkp_filename)
bkp_data = pd.read_json(path, lines=True)
bkp_data.head(5)

,_id,topic,payload,timestamp
0,{'$oid': '5d274101febb1ac74b9ce573'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,{'responseStamp': 1},2019-07-11 14:00:33.384
1,{'$oid': '5d274101febb1ac74b9ce574'},data/theme/zombie.case/get,{'responseStamp': 2},2019-07-11 14:00:33.470
2,{'$oid': '5d274101febb1ac74b9ce575'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,"{'name': 'Zombie Venom 01', 'source': '# 1 (ca...",2019-07-11 14:00:33.475
3,{'$oid': '5d274101febb1ac74b9ce576'},data/theme/zombie.case/get/response/2,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 14:00:33.786
4,{'$oid': '5d274101febb1ac74b9ce577'},data/theme/zombie.start/get,{'responseStamp': 3},2019-07-11 14:00:33.789


In [87]:
# Filtrar time stamp para 11/07/2019 das 11 as 14 GMT-03:00
data_session_11_07 = bkp_data.loc[lambda df: df.timestamp > '2019-07-11 14:00:00']
data_session_11_07 = data_session_11_07.loc[lambda df: df.timestamp < '2019-07-11 17:00:00']
data_session_11_07
# data_venom.to_json("executions-experiments-zv-group1.json", orient='split')

,_id,topic,payload,timestamp
0,{'$oid': '5d274101febb1ac74b9ce573'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,{'responseStamp': 1},2019-07-11 14:00:33.384
1,{'$oid': '5d274101febb1ac74b9ce574'},data/theme/zombie.case/get,{'responseStamp': 2},2019-07-11 14:00:33.470
2,{'$oid': '5d274101febb1ac74b9ce575'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,"{'name': 'Zombie Venom 01', 'source': '# 1 (ca...",2019-07-11 14:00:33.475
3,{'$oid': '5d274101febb1ac74b9ce576'},data/theme/zombie.case/get/response/2,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 14:00:33.786
4,{'$oid': '5d274101febb1ac74b9ce577'},data/theme/zombie.start/get,{'responseStamp': 3},2019-07-11 14:00:33.789
...,...,...,...,...
17623,{'$oid': '5d276592febb1ac74b9d2f5e'},control/knot/Início/selected,{'content': ''},2019-07-11 16:36:34.239
17624,{'$oid': '5d276592febb1ac74b9d2f5f'},data/theme/minimal.picture_up_left/get,{'responseStamp': 13},2019-07-11 16:36:34.244
17625,{'$oid': '5d276595febb1ac74b9d2f60'},data/theme/minimal.picture_up_left/get/respons...,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 16:36:37.056
17626,{'$oid': '5d276595febb1ac74b9d2f61'},data/template/Cancel/get,{'responseStamp': 14},2019-07-11 16:36:37.059


In [91]:
# Filtrar time stamp para 15/07/2019 das 10 as 14 GMT-03:00
data_session_15_07 = bkp_data.loc[lambda df: df.timestamp > '2019-07-15 12:00:00']
data_session_15_07 = data_session_15_07.loc[lambda df: df.timestamp < '2019-07-15 17:00:00']
data_session_15_07

,_id,topic,payload,timestamp
18439,{'$oid': '5d2c6efb739d8a1fc75aa8c1'},data/user/login,"{'email': 'jacinto@example.com', 'password': '...",2019-07-15 12:18:03.515
18440,{'$oid': '5d2c6efb739d8a1fc75aa8c2'},data/user/login/response/1,"{'userid': 1, 'token': 'eyJhbGciOiJIUzI1NiIsIn...",2019-07-15 12:18:03.859
18441,{'$oid': '5d2c6efb739d8a1fc75aa8c3'},data/case/*/list,"{'filterBy': 'user', 'filter': 1, 'responseSta...",2019-07-15 12:18:03.874
18442,{'$oid': '5d2c6eff739d8a1fc75aa8c4'},data/case/*/list/response/2,[{'id': '024c72c2-7251-4ff5-b848-65585fc362ee'...,2019-07-15 12:18:07.314
18443,{'$oid': '5d2c6f07739d8a1fc75aa8c5'},data/case/024c72c2-7251-4ff5-b848-65585fc362ee...,{'responseStamp': 3},2019-07-15 12:18:15.782
...,...,...,...,...
31541,{'$oid': '5d2c9276739d8a1fc75ae094'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'track': {'userid': 23, 'caseid': '09e94ada-6...",2019-07-15 14:49:26.348
31542,{'$oid': '5d2c9276739d8a1fc75ae095'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'responseStamp': 33, 'track': {'userid': 23, ...",2019-07-15 14:49:26.352
31543,{'$oid': '5d2c9276739d8a1fc75ae096'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'content': 'Previous Knot', 'track': {'userid...",2019-07-15 14:49:26.357
31544,{'$oid': '5d2c9276739d8a1fc75ae097'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-15 14:49:26.361


## Reconstructing user track

In [92]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        if 'track' in payload and 'userid' in payload['track']:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
    return users_tracks

In [120]:
def users_points_history(users_track, get_raw=False):
    users_points = dict()
    for user in users_track.keys():
        for log in users_track[user]:
            if 'var/points/set' in log['topic']:
#               print(log['payload'])
                points=log['payload']['content']
                if get_raw:
                    case_id = log['payload']['track']['caseid']
                    if user in users_points:
                        users_points[user].append((points, case_id))
                    else:
                        users_points.update({user: [(points, case_id)]})
                else:
                    if user in users_points:
                        users_points[user].append(points)
                    else:
                        users_points.update({user: [points]})
    return users_points

In [94]:
def dic_to_dataFrames(dictionary):
    data_frames = []
    for key in dictionary.keys():
        df = pd.DataFrame(dictionary[key],columns=[key])
        data_frames.append(df)

    return pd.concat(data_frames, axis=1)

In [95]:
def count_hit_and_miss(users_track):
    hits_misses = dict()
    for user in users_track.keys():
        hits   = 0
        misses = 0
        for log in users_track[user]:
            if  re.search('knot/\w+_certo/start', log['topic']):
                hits += 1
            if  re.search('knot/\w+_errado/start', log['topic']):
                misses += 1
        hits_misses[user] = {'hits':hits, 'misses':misses}
    return hits_misses

## Processamento de grupos

In [98]:
session_11_07_track = reconstruct_users_track(data_session_11_07)

In [97]:
session_11_07_track.keys()

dict_keys([21, 1, 17, 19, 16, 23, 22, 20, 18, 27, 31, 26, 32, 28, 30, 29, 24])

In [99]:
session_15_07_track = reconstruct_users_track(data_session_15_07)

In [100]:
session_15_07_track.keys()

dict_keys([1, 7, 9, 21, 10, 12, 11, 19, 14, 18, 16, 17, 26, 25, 23, 24])

In [124]:
user_points_session_11_07 = users_points_history(session_11_07_track)
for k in user_points_session_11_07:
    if user_points_session_11_07[k][0] != 100:
        user_points_session_11_07[k].insert(0, 100)

In [125]:
user_points_session_11_07 = dic_to_dataFrames(user_points_session_11_07)
user_points_session_11_07

,21,1,17,19,16,23,22,20,18,27,31,26,32,28,30,29,24
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0
1,90.0,100.0,90.0,90.0,110.0,90.0,90.0,90.0,90.0,110.0,90.0,110,90.0,90.0,110.0,110.0,90.0
2,80.0,NaN,80.0,80.0,100.0,80.0,100.0,80.0,80.0,100.0,100.0,100,100.0,100.0,100.0,120.0,100.0
3,90.0,NaN,90.0,90.0,90.0,70.0,110.0,70.0,70.0,110.0,90.0,90,90.0,90.0,110.0,110.0,90.0
4,80.0,NaN,100.0,80.0,100.0,60.0,100.0,80.0,60.0,100.0,80.0,100,100.0,80.0,100.0,100.0,100.0
5,90.0,NaN,90.0,70.0,110.0,70.0,110.0,90.0,50.0,90.0,70.0,110,90.0,90.0,110.0,110.0,90.0
6,80.0,NaN,100.0,80.0,100.0,80.0,120.0,80.0,60.0,80.0,80.0,100,80.0,100.0,100.0,100.0,80.0
7,90.0,NaN,90.0,90.0,90.0,90.0,110.0,90.0,70.0,70.0,70.0,90,70.0,NaN,90.0,110.0,90.0
8,100.0,NaN,80.0,100.0,100.0,80.0,120.0,80.0,80.0,60.0,60.0,80,60.0,NaN,100.0,120.0,100.0
9,110.0,NaN,70.0,110.0,110.0,70.0,110.0,90.0,90.0,NaN,70.0,70,50.0,NaN,90.0,110.0,110.0
